In [ ]:
!pip install PyMuPDF python-docx openai langchain langchain-openai crewai-tools
!pip install git+https://github.com/joaomdmoura/crewai.git

In [ ]:
# 📂 Step 2: Upload a Resume File (PDF or DOCX)
from google.colab import files
uploaded = files.upload()

resume_filename = list(uploaded.keys())[0]
print(f"✅ Uploaded file: {resume_filename}")

In [ ]:
import os
import fitz  # PyMuPDF
import docx  # python-docx

from langchain_openai import ChatOpenAI
from crewai import Agent, Task, Crew

# 🔐 Step 4: Set API Keys
# REPLACE with your actual API keys here or use environment variables
os.environ["OPENAI_API_KEY"] = ""

openai_api_key = os.environ.get("OPENAI_API_KEY")

# 💬 Step 5: Initialize the OpenAI model
llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=openai_api_key)

# 🧾 Step 6: Resume Parsing Functions
def extract_text_from_pdf(file_path):
    doc = fitz.open(file_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

def extract_text_from_docx(file_path):
    doc = docx.Document(file_path)
    return "\n".join([para.text for para in doc.paragraphs])

def extract_text_from_resume(file_path):
    if file_path.endswith(".pdf"):
        return extract_text_from_pdf(file_path)
    elif file_path.endswith(".docx"):
        return extract_text_from_docx(file_path)
    else:
        return "Unsupported file format"

# 🧠 Step 7: Read and Extract Resume Text
resume_text = extract_text_from_resume(resume_filename)
print("📄 Resume text extracted:")
print(resume_text[:1000])

In [ ]:
from crewai import Agent, Task, Crew

resume_feedback = Agent(
    role="Professional Resume Advisor",
    goal="Give feedback on the resume to make it stand out in the job market.",
    backstory="With a strategic mind and an eye for detail, you excel at providing feedback on resumes to highlight the most relevant skills and experiences.",
    verbose=True,
    llm=llm
)

resume_advisor = Agent(
    role="Professional Resume Writer",
    goal="Rewrite the resume to highlight the most relevant skills and experience.",
    backstory="With a keen eye for talent presentation, you craft resumes that grab attention and reflect a candidate's true potential.",
    verbose=True,
    llm=llm
)

resume_feedback_task = Task(
    description=(
        "Give feedback on the resume to make it stand out for recruiters. "
        "Review every section, including the summary, work experience, skills, and education. "
        "Suggest adding relevant sections if they are missing. Also give an overall score to the resume out of 10. "
        "This is the resume: {resume}"
    ),
    expected_output="The overall score of the resume followed by the feedback in bullet points.",
    agent=resume_feedback
)

resume_advisor_task = Task(
    description=(
        "Rewrite the resume based on the feedback to make it stand out for recruiters. "
        "You can adjust and enhance the resume but don't make up facts. Review every section to reflect the candidate's abilities. "
        "This is the resume: {resume}"
    ),
    expected_output="Resume in markdown format that effectively highlights the candidate's qualifications and experiences.",
    context=[resume_feedback_task],
    agent=resume_advisor
)

# 🎬 Step 9: Run the CrewAI Workflow
crew = Crew(
    agents=[resume_feedback, resume_advisor],
    tasks=[resume_feedback_task, resume_advisor_task],
    verbose=True
)

result = crew.kickoff(inputs={"resume": resume_text})

# ✅ Final Output
print("\n📝 Final Resume Output:\n")
print(result)

In [ ]:
with open("resume.md", "w") as f:
    f.write(str(result))

In [ ]:
!pip install pypandoc
!apt-get install pandoc
!apt-get install -y pandoc texlive-xetex texlive-fonts-recommended texlive-generic-recommended

In [ ]:
!apt-get install -y texlive-xetex texlive-fonts-recommended texlive-latex-extra

In [ ]:
import pypandoc

output = pypandoc.convert_file(
    "resume.md",
    "pdf",
    outputfile="resume.pdf",
    extra_args=["--pdf-engine=xelatex", "--variable", "geometry:margin=1in"]
)
print("✅ PDF resume created!")

In [ ]:
from google.colab import files
files.download("resume.pdf")